<p style="font-size:300%; text-align:center"> king county housing price modeling</p>
<p style="font-size:150%; text-align:center"> validation</p>

In [2]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium 
plt.style.use('ggplot')

##  Read the clean normalized KC housing data

In [3]:
kc = pd.read_csv('data/kc_house_data_normalized.csv')
kc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21398 entries, 0 to 21397
Data columns (total 21 columns):
id               21398 non-null float64
bedrooms         21398 non-null float64
bathrooms        21398 non-null float64
sqft_living      21398 non-null float64
sqft_lot         21398 non-null float64
floors           21398 non-null float64
view             21398 non-null float64
condition        21398 non-null float64
grade            21398 non-null float64
sqft_basement    21398 non-null float64
yr_built         21398 non-null float64
sqft_living15    21398 non-null float64
sqft_lot15       21398 non-null float64
r                21398 non-null float64
waterfront_0     21398 non-null float64
waterfront_1     21398 non-null float64
season_winter    21398 non-null float64
season_spring    21398 non-null float64
season_summer    21398 non-null float64
season_fall      21398 non-null float64
price            21398 non-null float64
dtypes: float64(21)
memory usage: 3.4 MB


In [5]:
'''
let\'s make scikit learns test train splitting'
'''
#separating independent and dependent variable
X = kc.iloc[:,1:].values
y = kc.iloc[:,0].values
#splitting dataset into training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state = 0)


In [6]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [9]:
#Backward Elimination
import statsmodels.formula.api as sm
#import statsmodels.api as sm
from statsmodels.formula.api import ols


def backwardElimination(x, SL):
    numVars = len(x[0])
    temp = np.zeros((21398,19)).astype(int)
    for i in range(0, numVars):
        regressor_OLS = ols(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if maxVar > SL:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, 1)
                    tmp_regressor = sm.OLS(y, x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    regressor_OLS.summary()
    return x
 
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17]]
X_Modeled = backwardElimination(X_opt, SL)

PatsyError: model is missing required outcome variables